# Еще немного про визуализацию графов в Azure

> Этот ноутбук работает на Linux, а значит должно работать и в [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/eosfor/scripting-notes/HEAD)

## Сценарий

Предположим, что у нас есть некая сетевая структура - звезда. Она динамичная, сетки на периферии добавляются и исчезают динамически и нам заранее не известно сколько их есть на данный момент. Первое, что мы хотим сделать - визуализировать топологию сети. Так же мы хотим подсвечивать сети, пиринг на которых не сконфигурирован до конца, поскольку мы знаем что такая проблема иногда возникает из-за ошибок в автоматизации создания сетей. И наконец, к сетям подключены устройства, и мы знаем что одно из них странно себя ведет - где-то работает, а где-то нет. Мы хотим посмотреть, в каких сетях эти устройства для дальнейшего анализа и идентификации проблемы.

## Подготовка

В этой секции мы подготовим наш стенд. Создадим сети, пиринги между ними. Эмулируем ошибку, а так же добавим сетевые интерфейсы - они сыграют роль виртуальных машин в этих сетях

In [ ]:
Install-Module -Name PSQuickGraph -AllowPrerelease -RequiredVersion "2.0.2-alpha"

Загрузим нужные модули

In [ ]:
Import-Module ipmgmt
Import-Module PSQuickGraph -RequiredVersion "2.0.2"

Авторизуемся в Azure

In [ ]:
Login-AzAccount

Зададим начальные константы

In [ ]:
$rgName = "vnet-test"
$region = "eastus2"
$baseNet = "10.96.0.0/16"

Создадим ресурсную группу, если еще нет такой. Чтоб два раза не вставать и в портал не ходить

In [ ]:
New-AzResourceGroup -Name $rgName -Location $region

Подготовим параметры для создания сетей. Здесь мы используем модуль `ipmgmt`, который рассматривали в [передыдущем](https://github.com/eosfor/scripting-notes/blob/main/notebooks/ru/ipmgmt.ipynb) ноутбуке. Мы просто формируем массив из хеш таблиц. Этот массив, можно использовать в команде `Get-VLSMBreakdown`, чтобы рассчитать диапазоны сеток, которые мы хотим создать. Мы просто итерируем по алфавиту от "A" до "K", для каждой буквы создаем соответствующую хеш таблицу. В ней есть имя сети и ее размер.

In [ ]:
$vnets = "A".."K" | % { @{type = "VNET-$_"; size = (256-2)} }

Теперь просто передаем этот массив в команду, которая и посчитает нам разбиение сети на подсети. Нам надо отфильтровать зарезервированные, чтобы не сбивало с толку.

In [ ]:
Get-VLSMBreakdown -Network $baseNet -SubnetSize $vnets | ? type -ne 'reserved' | 
    ft type, network, netmask, *usable, cidr -AutoSize


type   Network    Netmask       FirstUsable LastUsable   Usable Cidr
----   -------    -------       ----------- ----------   ------ ----
VNET-K 10.96.10.0 255.255.255.0 10.96.10.1  10.96.10.254    254   24
VNET-J 10.96.9.0  255.255.255.0 10.96.9.1   10.96.9.254     254   24
VNET-I 10.96.8.0  255.255.255.0 10.96.8.1   10.96.8.254     254   24
VNET-H 10.96.7.0  255.255.255.0 10.96.7.1   10.96.7.254     254   24
VNET-G 10.96.6.0  255.255.255.0 10.96.6.1   10.96.6.254     254   24
VNET-F 10.96.5.0  255.255.255.0 10.96.5.1   10.96.5.254     254   24
VNET-E 10.96.4.0  255.255.255.0 10.96.4.1   10.96.4.254     254   24
VNET-D 10.96.3.0  255.255.255.0 10.96.3.1   10.96.3.254     254   24
VNET-C 10.96.2.0  255.255.255.0 10.96.2.1   10.96.2.254     254   24
VNET-B 10.96.1.0  255.255.255.0 10.96.1.1   10.96.1.254     254   24
VNET-A 10.96.0.0  255.255.255.0 10.96.0.1   10.96.0.254     254   24



Теперь можно эти сети создать. В каждой сети одна подсеть, для простоты. Но ничто не мешает использовать `Get-VLSMBreakdown` еще раз, автоматически посчитать подсети и каждую из них разделить как вам надо.

In [ ]:
Get-VLSMBreakdown -Network $baseNet -SubnetSize $vnets | ? type -ne 'reserved' | % {
    $addressPrefix = "$($_.network)/$($_.cidr)"
    $subnet = New-AzVirtualNetworkSubnetConfig -Name "default" -AddressPrefix $addressPrefix
    New-AzVirtualNetwork -Name $_.type -ResourceGroupName $rgName -Location $region -AddressPrefix $addressPrefix -Subnet $subnet
}

Теперь добавим VNET Peering

In [ ]:
$nets = Get-AzVirtualNetwork -ResourceGroupName $rgName

Построим hub-and-spoke топологию. В данном случае нам все равно какая из сеток будет хабом. Потому возьмем нулевую в качестве hub, остальные - spokes. Ну и пропустим один обратный пиринг, почему бы и нет ;)

In [ ]:
$hub = $nets[0]
$spokes = $nets[1..($nets.count-1)]

$spokes | % {
    Add-AzVirtualNetworkPeering `
        -Name "$($hub.Name)-$($_.Name)" `
        -VirtualNetwork $hub `
        -RemoteVirtualNetworkId $_.Id
}

$spokes | select -Skip 1 |  % {      
    Add-AzVirtualNetworkPeering `
        -Name "$($_.Name)-$($hub.Name)" `
        -VirtualNetwork $_ `
        -RemoteVirtualNetworkId $hub.Id
}

Для моделирования устройств, чтобы не создавать виртуальных машин просто добавим пару-тройку сетевух в случайные сеточки. Ну, не совсем в случайные. Тут есть пример хитрого синтаксиса для командлета `... | % {} {} {}`. Командлет `Foreach-Object` принимает три скриптблока `begin`, `process`, `end`. В данном случае мы инициализируем переменную в `begin` блоке, а затем используем ее в `process`. Таким образом она остается локальной внутри командлета, но вместе с тем оказывается проинициализиованной. Таким образом нулевая сеть в списке `spokes` всегда будет содержать сетевой интерфейс. Остальные расположатся случайным образом

In [ ]:
1..4 | % {$idx = 0} {
    $vnetForNIC = $spokes[$idx]
    New-AzNetworkInterface -Name "NetworkInterface-$idx" -ResourceGroupName $rgName -Location $region -SubnetId $vnetForNIC.Subnets[0].Id
    $idx = Get-Random -Minimum 1 -Maximum ($spokes.Count-1)
}

## Игра

Если все прошло нормально - мы готовы к эксперименту. Прочитаем все сеточки из нашей ресурсной группы, ну и за одно создадим пустой граф

In [ ]:
$vnets = Get-AzVirtualNetwork -ResourceGroupName $rgName
$g = New-Graph

Граф нам придется заполнять в несколько проходов. В первый проход мы добавим сети, затем пиринги и, наконец, сетевые интерфейсы.

#### Первая часть "Марлезонского балета" - VNETs

У нас есть все необходимое, чтобы добавить в граф сети. Однако есть одна небольшя проблема. В граф можно добавлять любые объекты. При этом добавляемый объект преобразуется в специальный тип `PSGraph.Model.PSVertex`. Одним из важнейших свойств в этом типе является свойство `Label`, которое используется для сравнения вершин при добавлении, поиске и визуализации. Кроме того, этот тип содержит свойство `OriginalObject`, которое в свою очередь хранит целый объект, из которого получается эта вершина. По сути граф представляет собой упрощенную *in-memory* графовую базу данных, поскольку хранит все объекты, которые вы в него передали. Для того чтобы автоматически заполнить поле `Label` у каждого объекта, который вы передаете в граф, вызывается метод `ToString()`. Однако для объектов типа `PSVirtualNetwork` вызов `ToString()` возвращает только имя типа, что делает все `Label` всех вершин графа одинаковыми. Нам же необходимо, чтобы каждый уникальный объект имел свою уникальную `Label`. Для этого есть другой способ. У `PSGraph.Model.PSVertex` есть конструктор, который принимает `Label` и сам оригинальный объект. Таким образом мы можем переопределить `Label` в нашем коде, и просто передать уже готовое значение, и в дополнение к ней передеать сам оригинальный объект. Это мы и делаем ниже. В качестве метки выбран `resourceID` виртуальной сети, вторым параметром передается сам объект сети.

> При экспорте в `dot` или `GraphML` данные из `OriginalObject` теряются.

In [ ]:
$vnets | % {
    Add-Vertex -Graph $g -Vertex ([PSGraph.Model.PSVertex]::new($_.Id, $_))
}

#### Вторая часть "Марлезонского балета" - VNET peerings

В этот момент у нас в графе уже есть сети, но нет связей между ними - network peering. Чтобы их добавить мы побежим по самому графу - по всем вершинам, которые мы только что в него добавили. Как уже упоминалось, какждая вершина, в своем свойстве `OriginalObject` хранит оригинальный объект, из которого она создалась. В нашем случае это объект `PSVirtualNetwork`, мы его передали вторым параметром. Соотвественно, нам нужно для каждой вершины графа заглянуть в свойство `OriginalObject`, в котором хранится сеть, и в ней уже для каждого peering посмотреть, накакую сеть он показывает. Эта сеть хранится в виде `resourceID`. Мы предполагаем, что все возможные сети уже есть в графе, мы их добавили на предыдущем шаге. Это означает, что имея этот `resourceID` мы, в этом же графе можем найти вершину, являющуюся второй частью peering, и создать между ними ребро.

Простыми словами - мы бежим по графу, вершинами которого являются сети. Для каждой вершины смотрим, с кем настроен ее peering. Используя `resourceID` из этого peering находим в графе вторую сеть и связываем их ребром.

In [ ]:
foreach ($v in $g.Vertices){
    foreach($p in $v.OriginalObject.VirtualNetworkPeerings) {
        foreach ($rvn in $p.RemoteVirtualNetwork) {
            $targetVertex = $g.Vertices.Where({$_.Label -eq $rvn.id})[0]
            Add-Edge -From $v -To $targetVertex -Graph $g
        }
    }
}

В этот момент у нас в графе есть сетки и связи между ними. И это решает нашу первую задачу - нарисовать граф. Теперь мы хотим подсветить те сети, у которых не полностью сконфигурирован пиринг. Здесь мы немного схитрим, конечно. Мы знаем как именно он "неправильно" сконфигурирован - у него осутствует исходящая связь. Поэтому пробежимся во всем вершинам еще раз, и покрасим цветом все те, у которых отсуствует исходящая связь. Мы можем точно так же покрасить и те, у которых нет входящих связей - но для простоты опустим этот момент

In [ ]:
$g.Vertices | % { if ( $g.OutDegree($_) -eq 0 ) 
                    { $_.GVertexParameters.Fillcolor = [QuikGraph.Graphviz.Dot.GraphvizColor]::OrangeRed  } 
                }

#### Третья часть "Марлезонского балета" - NICs

На данном этапе в нашем графе присутствует топология сети, и покрашены "поврежденные" вершины. Пришло время добавить сетевые интерфейсы. Прочитаем их для начала

In [ ]:
$nics = Get-AzNetworkInterface -ResourceGroupName $rgName

В свойстве `IpConfigurations` каждого интерфейса указана подсеть, в которую он подключен, в виде `resourceID`. И частью этого `resourceID` является и `resourceID` той виртуальной сети, в которой эта подсеть находится. Таким образом нам надо просто отрезать все, начиная с `/subnets/` и до конца строки, что мы и делаем регулярным выражением. А дальше все, как мы делали выше - находим в том же графе нужную сеть, и добавляем ребро между интерфейсом и сеткой.

In [ ]:
$nics | % {
    $vnetID = $_.IpConfigurations[0].Subnet.Id -replace "/subnets/.+", ""
    $targetVertex = $g.Vertices.Where({$_.Label -eq $vnetID})[0]
    Add-Edge -Graph $g -From ([PSGraph.Model.PSVertex]::new($_.name, $_)) -To $targetVertex
}

теперь в графе есть объекты двух типов - сети и сетевые интерфейсы. Мы хотим покрасить **только** сетевые интерфейсы, с тем, чтобы они выделялись в визуализации. Для этого мы побежим по вершинам графа еще раз, заглянем в каждую, в свойство `OriginalObject` и проверим его тип. Если это `Microsoft.Azure.Commands.Network.Models.PSNetworkInterface` - покрасим в соответствующий цвет

In [ ]:
$g.Vertices | 
    ? {$_.OriginalObject.GetType() -eq [Microsoft.Azure.Commands.Network.Models.PSNetworkInterface]} | 
    % {  $_.GVertexParameters.Fillcolor = [QuikGraph.Graphviz.Dot.GraphvizColor]::WhiteSmoke }

#### Экспорт и визуализация

Нам осталось только экспортнуть граф, как мы это делали [раньше](https://github.com/eosfor/scripting-notes/blob/main/notebooks/ru/analyze-sysmon-events.ipynb).

In [ ]:
Export-Graph -Graph $g -Path "$($env:TEMP)\topology.svg" -Format MSAGL_MDS
Export-Graph -Graph $g -Path "$($env:TEMP)\topology.gv" -Format Graphviz

In [ ]:
using System.IO;

var path = Path.GetTempPath();
var svg = File.ReadAllText($"{path}\\topology.svg");
var gv = File.ReadAllText($"{path}\\topology.gv");
/// svg.DisplayAs("text/html"); // эта строка покажет большую картинку, которая не влезает в секцию вывода

In [ ]:
#!html
<div id="mynetwork" style="height: 800px;"></div>

И отрисовать его в vis.js

In [ ]:
#!js
#!share --from csharp gv

visRequire = interactive.configureRequire({
    paths: {
        visjs: "https://visjs.github.io/vis-network/standalone/umd/vis-network.min"
    }
});
    
visRequire(["visjs"], visjs => {
    
    var container = document.getElementById("mynetwork");
    var dot = gv;
    var parsedData = visjs.parseDOTNetwork(dot);

    var data = {
        nodes: parsedData.nodes,
        edges: parsedData.edges
    };
    var options = parsedData.options;
    options = {
        physics: {
            solver: "forceAtlas2Based",
            enabled: false,
            forceAtlas2Based: {
                theta: 0.5,
                gravitationalConstant: -50,
                centralGravity: 0.01,
                springConstant: 0.08,
                springLength: 100,
                damping: 0.4,
                avoidOverlap: 0.7
              },
            barnesHut: {
                theta: 0.5,
                gravitationalConstant: -2000,
                centralGravity: 0.3,
                springLength: 95,
                springConstant: 0.04,
                damping: 0.09,
                avoidOverlap: 0
            }
        },
        interaction: { hover: true, zoomView: true },
        layout: { randomSeed: 'Mickey' }
    }

    var network = new visjs.Network(container, data, options); 
    network.stabilize(600)
});

Если теперь присмотреться к картинке, то вы увидите, что "поврежденная" сеть покрашена красным цветом, и к ней присоединен один из сетевых интерфейсов. Это, собственно и является потенциальной причиной проблем на этом устройстве.